In [1]:
import rasterio
import numpy as np
import os
# from matplotlib import pyplot as plt
# %matplotlib inline

In [2]:
ACQ_NAME = 'portland-metro_2014'

In [3]:
COL_ORDER = ['strat0_return-proportion','strat1_return-proportion','strat2_return-proportion',
             'strat3_return-proportion','strat4_return-proportion','strat5_return-proportion',
             'strat0_intensity-median','strat1_intensity-median','strat2_intensity-median',
             'strat3_intensity-median','strat4_intensity-median','strat5_intensity-median',
             'height_05-percentile','height_25-percentile','height_50-percentile',
             'height_75-percentile','height_95_percentile','height_max','cover',
             'potential_volume','stddev_height','surface_area_ratio','surface_volume',
             'surface_volume_ratio','aspect','elevation','overall_curvature','plan_curvature',
             'profile_curvature','slope','solar_radiation_index']

In [4]:
LOOK_IN = '../data/interim/lidar/processed_lidar_acquisitions/lidar_derived_rasters/{}'.format(ACQ_NAME)
for col in COL_ORDER:
    if not os.path.exists(os.path.join(LOOK_IN, col + '.tif')):
        print('Unable to find {}'.format(col))

In [5]:
with rasterio.open(os.path.join(LOOK_IN, col+'.tif')) as src:
    img = src.read(1, masked=True)

In [6]:
STACK_RASTER = '../data/processed/lidar/processed_lidar_acquisitions/lidar_derived_rasters/{}.tif'.format(ACQ_NAME)
stack_meta = src.meta.copy()
num_layers = len(COL_ORDER)
stack_meta.update(count=num_layers,
                  bigtiff='YES',
                  compress='LZW',
                  num_threads='ALL_CPUS'
                 )

with rasterio.open(STACK_RASTER, 'w', **stack_meta) as dst:
    for i in range(num_layers):
        to_open = os.path.join(LOOK_IN, COL_ORDER[i] + '.tif')
        with rasterio.open(to_open) as src:
            img = src.read(1, masked=True)
        dst.write_band(i+1, img)
        dst.set_band_description(i+1, COL_ORDER[i])

In [7]:
# with rasterio.open(STACK_RASTER) as src:
#     img = src.read(masked=True)

In [8]:
# img.shape